In [1]:
import pandas as pd
import numpy as np
import os
import cv2

In [2]:
df = pd.read_csv("./open/train.csv")

In [3]:
def get_ratio(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    img_area = shape[0] * shape[1]
    return sum(lengths) / img_area * 100, (img_area - sum(lengths)) / img_area * 100

In [5]:
building_ratio_lst = []
background_ratio_lst = []
for idx, row in df.iterrows():
  img_path = df.iloc[idx, 1]
  img = cv2.imread(os.path.join("./open", img_path))
  h, w, _ = img.shape
  mask_rle = df.iloc[idx, 2]
  building_ratio, background_ratio = get_ratio(mask_rle, (h, w))
  building_ratio_lst.append(building_ratio)
  background_ratio_lst.append(background_ratio)

In [10]:
ratio_lst = []
for br in building_ratio_lst:
    if br <= 3:
        ratio_lst.append(3)
    elif br <= 5:
        ratio_lst.append(5)
    elif br <= 10:
        ratio_lst.append(10)
    elif br <= 30:
        ratio_lst.append(30)
    elif br <= 50:
        ratio_lst.append(50)
    else:
        ratio_lst.append(100)
df["ratio_type"] = ratio_lst
df.to_csv("train_ratio.csv", index=False)

In [11]:
from collections import Counter
Counter(ratio_lst)

Counter({10: 2432, 3: 2263, 5: 1349, 30: 1086, 50: 10})